<img src="https://toppng.com/uploads/preview/linkedin-logo-png-photo-116602552293wtc4qogql.png" width="20" height="20" /> [Bharath Hemachandran](https://www.linkedin.com/in/bharath-hemachandran/)

# 🤖 Phase 1: One Groq API Call (No MCP)

Learn **prompt engineering** basics, how **parameters** change the model's output, and how to add **context** to your prompts. Same encode → vectors → decode as Phase 0 happens on the server; here we focus on **prompts**, **temperature**, **top_p**, **max_output_tokens**, and **instructions**.

<div style="background: #e3f2fd; padding: 14px; border-radius: 8px; border-left: 4px solid #1976d2;">
<strong>🎯 What you'll do:</strong> Prompt basics, parameter tuning (how it changes output), common activities for the right settings, and how to include more context in your prompt.
</div>

### 📋 Notebook objective (table of contents)

This notebook covers:
- **Setup** — Install OpenAI client (Groq-compatible), API key
- **Basics of prompt engineering** — What is a prompt; input vs instructions; clarity, role, task, format
- **Parameters** — temperature, top_p, max_output_tokens, truncation, instructions
- **How tuning parameters changes output** — Temperature, length, instructions (with examples)
- **API call** — Single Groq Responses API request
- **Token usage** — input_tokens, output_tokens (with a simple chart)
- **Output text** — Model reply
- **Common activities for the right settings** — Checklist: factual vs creative, length, instructions, iteration
- **Including more context in your prompt** — Longer input, few-shot examples, structure, instructions
- **Try it yourself** — Suggestions to tweak prompts and parameters
- **Exercises** — Factual answers, structured prompts, shortening replies
- **Additional reading** — Videos and blogs


## 🔧 Setup (run once)

Install **openai** (Groq is OpenAI-compatible). On Colab, run this cell first.

In [1]:
!pip install -q openai matplotlib


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


### 🔑 Set your Groq API key

Get a free key at [console.groq.com](https://console.groq.com/keys). In Colab you can use **Secrets** or run the cell below and paste when prompted.

In [2]:
import os
from getpass import getpass

if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass("Paste your GROQ_API_KEY: ")

from openai import OpenAI

def get_groq_client():
    return OpenAI(
        api_key=os.environ["GROQ_API_KEY"],
        base_url="https://api.groq.com/openai/v1",
    )

print("✅ Groq client ready.")

✅ Groq client ready.


## ✍️ Basics of prompt engineering

A **prompt** is the text you send to the model. Good prompts are **clear**, **specific**, and give the model a **role**, **task**, and (optionally) **format**.

### Input vs instructions

- **`input`** — The main user message (the question or request). This is what the model “sees” as the current turn.
- **`instructions`** — Optional **system** message: tone, role, or global rules (e.g. “You are a helpful assistant. Answer in one short sentence.”). The model treats this as background context for the whole conversation.

Use **instructions** for *how* the model should behave (role, tone, length). Use **input** for *what* you’re asking (the actual question or task).

### What makes a good prompt?

1. **Role** — “You are a Python tutor” / “You are a summarizer” so the model knows the style.
2. **Task** — Say exactly what you want: “Summarize the following in 2 sentences” vs “Summarize this.”
3. **Format** — “Reply with a bullet list” / “One sentence only” / “JSON with keys: title, summary.”
4. **Context** — Put relevant facts, documents, or examples *in* the prompt (or in instructions) so the model has something to work with.

Below we’ll use a simple **input** and optional **instructions**; later we’ll add more context and structure.

In [3]:
# Example: same task with a vague vs a clear prompt (we'll call the API with the clear one later)
vague = "Tell me about MCP"
clear = "In one sentence, what is the Model Context Protocol? Explain for a developer."

print("Vague prompt:", repr(vague))
print("Clear prompt:", repr(clear))
print("\nClear prompt specifies: task (one sentence), topic (MCP), audience (developer).")

Vague prompt: 'Tell me about MCP'
Clear prompt: 'In one sentence, what is the Model Context Protocol? Explain for a developer.'

Clear prompt specifies: task (one sentence), topic (MCP), audience (developer).


## 🎛️ Parameters you can change

<div style="background: #fff8e1; padding: 12px; border-radius: 8px;">
<strong>Sampling:</strong> <code>temperature</code> (0 = deterministic, 2 = very random), <code>top_p</code> (nucleus sampling).<br>
<strong>Length:</strong> <code>max_output_tokens</code> caps the reply length.<br>
<strong>Context:</strong> <code>truncation</code> = "auto" trims long inputs; "disabled" = no trim.<br>
<strong>Behavior:</strong> <code>instructions</code> (system) fixes tone or task.
</div>

### How tuning parameters changes the output

| Parameter | Low / strict | Effect | High / loose | Effect |
|-----------|----------------|--------|----------------|--------|
| **temperature** | 0 | Same prompt → same reply (deterministic). Best for facts, code, exact answers. | 0.8–1.2 | More variety, creativity; may be less consistent. Best for brainstorming, varied phrasing. |
| **top_p** | 0.1 | Only the most likely tokens (narrow). | 1.0 | No nucleus cutoff; use with temperature for diversity. |
| **max_output_tokens** | 50 | Short replies; good for one sentence or a list. | 500+ | Longer replies; risk of rambling if the task is vague. |
| **instructions** | "One sentence only." | Constrains style and length. | None | Model chooses length and style. |

**Practical rule of thumb:** Use **low temperature (0–0.3)** for factual, reproducible answers; **higher (0.7–1.0)** for creative or varied text. Set **max_output_tokens** to the length you need (e.g. 100 for a short summary). Use **instructions** to fix role, tone, and format so you don’t rely only on the user prompt.

In [7]:
# Compare output with low vs high temperature (same prompt)
client = get_groq_client()
prompt = "In one sentence, what is the Model Context Protocol?"

for temp, label in [(0.0, "Temperature 0 (deterministic)"), (2, "Temperature 2 (more random)")]:
    r = client.responses.create(
        model="llama-3.3-70b-versatile",
        input=prompt,
        temperature=temp,
        max_output_tokens=80,
    )
    print(f"--- {label} ---")
    print(r.output_text)
    print()

--- Temperature 0 (deterministic) ---
The Model Context Protocol is a proposed standard for describing and exchanging information about machine learning models, including their training data, performance metrics, and other relevant context, to facilitate transparency, explainability, and reproducibility.

--- Temperature 2 (more random) ---
The Model Context Protocol is a proposed standard for representing, communicating, and sharing modeling contexts, which facilitates collaboration, comparison, and re-use of conceptual models from multiple disciplines and organizations.



In [ ]:
client = get_groq_client()

TEMPERATURE = 0.7
TOP_P = 1.0
MAX_OUTPUT_TOKENS = 150
TRUNCATION = "disabled"
INSTRUCTIONS = None

kwargs = {
    "model": "llama-3.3-70b-versatile",
    "input": "In one sentence, what is the Model Context Protocol?",
    "temperature": TEMPERATURE,
    "top_p": TOP_P,
    "max_output_tokens": MAX_OUTPUT_TOKENS,
    "truncation": TRUNCATION,
}
if INSTRUCTIONS is not None:
    kwargs["instructions"] = INSTRUCTIONS

response = client.responses.create(**kwargs)
print(response)

## 📊 Token usage (tokenizing)

Input and output are counted in **tokens**, not characters—same idea as Phase 0.

In [ ]:
usage = getattr(response, "usage", None)
if usage:
    inp = getattr(usage, "input_tokens", 0)
    out = getattr(usage, "output_tokens", 0)
    tot = getattr(usage, "total_tokens", inp + out)
    print("📈 Token usage:")
    print(f"   input_tokens:  {inp}")
    print(f"   output_tokens: {out}")
    print(f"   total_tokens:  {tot}")
else:
    print("Usage not available.")

In [ ]:
import matplotlib.pyplot as plt

if usage:
    inp = getattr(usage, "input_tokens", 0)
    out = getattr(usage, "output_tokens", 0)
    fig, ax = plt.subplots(figsize=(5, 3))
    ax.bar(["Input", "Output"], [inp, out], color=["#1976d2", "#388e3c"], alpha=0.8)
    ax.set_ylabel("Tokens")
    ax.set_title("📊 Input vs output tokens")
    plt.tight_layout()
    plt.show()

## 🧪 Comparing models: size & specialization

Different models can give different answers to the **same prompt**:

- **Smaller / lower-parameter models** (often called *instant* / *fast*):
  - Faster, cheaper, great for simple tasks, drafts, or high-volume workloads.
  - May struggle more with complex reasoning or following subtle instructions.
- **Larger models** (more parameters, e.g. 70B):
  - Better at nuanced reasoning, complex instructions, and edge cases.
  - Slower and more expensive per token.
- **Specialized / tuned models (e.g. coding-tuned)**:
  - Trained or tuned specifically for code, chat, or other domains.
  - Often better at formatting, idiomatic style, and domain-specific tasks.

**Prompt differences between models:**
- Smaller or older models often need **more explicit instructions** (role, format, constraints).
- Code-tuned models may follow code-style instructions better (e.g. “PEP8-compliant”, “add docstring”, “return only code”).
- Safety / refusal behavior may differ; sometimes you need to be clearer about what is allowed or provide more benign context.

## 📤 Output text

In [ ]:
print(response.output_text)
print("\n--- raw output (first 400 chars) ---")
print(str(response.output)[:400])

## ✅ Common activities to get the right settings

Use this as a short checklist when tuning your call:

| Goal | What to do |
|------|------------|
| **Factual, reproducible answers** | Set **temperature** to 0 (or &lt; 0.3). Same prompt → same reply. |
| **Creative or varied text** | Use **temperature** 0.7–1.0. Try the same prompt twice to see variation. |
| **Short replies** | Set **max_output_tokens** (e.g. 50–100). Add **instructions** like “One sentence only.” |
| **Longer replies** | Increase **max_output_tokens** (e.g. 300–500). Be specific in the prompt so the model doesn’t ramble. |
| **Fixed tone or role** | Put it in **instructions** (e.g. “You are a concise technical writer.”). |
| **Stable format** | Ask in **input** or **instructions**: “Reply with a bullet list” / “JSON with keys: …” |
| **Check cost/length** | Read **usage** (input_tokens, output_tokens). Short prompts + low max_output_tokens = fewer tokens. |
| **Iterate** | If the output is wrong or noisy: clarify the prompt, add an example, or tighten instructions; then re-run. |

In [ ]:
# Compare different models on a small coding task
client = get_groq_client()

# Replace the second entry with a smaller or code-tuned model you have access to.
MODELS_TO_COMPARE = [
    ("llama-3.3-70b-versatile", "General 70B (versatile)"),
    ("llama-3.1-8b-instant", "Smaller / faster model (example)")
]

coding_prompt = (
    "Write a short Python function `is_valid_ipv4` that returns True if a string "
    "is a valid IPv4 address, otherwise False. Include a one-line docstring."
)

for model_id, label in MODELS_TO_COMPARE:
    print(f"--- {label} ({model_id}) ---")
    try:
        r = client.responses.create(
            model=model_id,
            input=coding_prompt,
            temperature=0,
            max_output_tokens=200,
        )
        print(r.output_text.strip())
    except Exception as e:
        print("Error calling model:", e)
    print()

## 📎 Including more context in your prompt

The model only “sees” what you send. More **relevant context** in the prompt usually improves answers.

### 1. Put context in `input`

- **Longer input** — Paste the document, article, or code you want summarized or questioned. The model uses it as context (subject to context-window limits).
- **Structure it** — Use headings, bullets, or labels so the model can find the right part: e.g. “Context: …” then “Question: …”.

### 2. Use `instructions` for role and rules

- **Role** — “You are a Python expert.” / “You are a summarizer for non-experts.”
- **Rules** — “Always answer in one short paragraph.” / “If unsure, say so.”
- **Format** — “Reply with a bullet list.” / “Output valid JSON only.”

### 3. Few-shot examples (in the prompt)

- Give 1–3 **example input → output** pairs in the **input** text. The model will tend to follow the same format or style.
- Example: “Example 1: … → Summary: … Example 2: … → Summary: … Now summarize: [your text]”

### 4. Truncation for long context

- If your **input** is very long, set **truncation** to `"auto"` so the API can trim it to fit the model’s context window. Otherwise the request may fail or the model may miss the end.

Below: an example with **instructions** (role + length) and a **structured input** (context + question).

In [ ]:
# Example: more context — instructions (role + format) + structured input (context + question)
client = get_groq_client()

instructions = (
    "You are a helpful assistant for developers. "
    "Answer in 1-2 short sentences. Be precise."
)
context = (
    "Context: The Model Context Protocol (MCP) is an open protocol that lets "
    "LLM applications connect to external tools and data sources in a standard way.\n\n"
    "Question: What is MCP and why would a developer use it?"
)

r = client.responses.create(
    model="llama-3.3-70b-versatile",
    input=context,
    instructions=instructions,
    temperature=0.3,
    max_output_tokens=80,
)
print("Instructions (system):", instructions[:60] + "...")
print("\nInput (structured):", context[:100] + "...")
print("\n--- Model reply ---")
print(r.output_text)

## ✏️ Try it yourself

<div style="background: #e8f5e9; padding: 12px; border-radius: 8px; border-left: 4px solid #4caf50;">
<strong>Prompts:</strong> Edit <code>input</code> in the API call cell: try a vague vs clear question; add "Context: …" then "Question: …"; or add 1–2 few-shot examples.<br>
<strong>Parameters:</strong> Set <code>TEMPERATURE</code> to 0 for deterministic output; to 0.9 for more variety. Set <code>MAX_OUTPUT_TOKENS</code> to 50 for a short reply or 300 for longer. Add <code>INSTRUCTIONS = "Answer in one short sentence only."</code> (or a role like "You are a Python tutor.") and re-run.<br>
<strong>Context:</strong> In the "Including more context" cell, change <code>instructions</code> or <code>context</code> and compare the reply.
</div>

In [ ]:
print("✅ Phase 1 complete. Next: Phase 2 (Groq + one MCP).")

## ✏️ Exercises

*Use only what you learned in this phase (prompts, parameters, instructions, context).*

1. **Factual, one-sentence answers**  
   You want the model to give **one short, factual answer** with no creativity or extra commentary. What would you set for `temperature` (and optionally `top_p`), and what would you put in `instructions`? Give concrete values or example text.

2. **Structured prompt**  
   Write a short prompt that includes a **"Context:"** block and a **"Question:"** block (you can use a toy context and question). In one or two sentences, explain why separating context and question helps the model.

3. **Long, rambling replies**  
   Suppose the model often returns long, rambling replies and you want shorter ones. Name **two** parameters or prompt changes you could make (from this chapter) and how each would help. No need to write code; just describe the knobs and the effect.

## 📚 Additional reading

**YouTube (verified)**  
- [Getting Started with Groq API](https://www.youtube.com/watch?v=S53BanCP14c) — Near real-time LLM chat with Groq.  
- [Groq API in Python](https://www.youtube.com/watch?v=jScpBCBoGdU) — Running generative AI with Groq (popular tutorial).

**Blogs (popular)**  
- [Groq API Reference](https://console.groq.com/docs) — Official docs: models, parameters, token usage.  
- [Sampling: temperature, top-k, top-p](https://huyenchip.com/2024/01/16/sampling.html) — Chip Huyen: generation configs explained.